## Part 2: Finding your way on a map

In the second part of the homework, we ask you to implement the core parts of a routing algorithm, including the visualization of the route on a map. In other words you will be making something that maybe could pass as a very early prototype of Google Maps. And although everything will be kind-of bare-bones and in a much smaller scale compared to the route planners you find on the net, the basic principles are much the same.

#### On grading

The homework is graded on a scale from 0 to 100. For each question we indicate how many points you can get. If the answer is not completely correct but nonetheless on the right track, we may decide to give partial credit.

For this and future homeworks we require **50 points** or more to pass. If you score below this threshold, the homework will count as failed. 


#### Required Python module

The visualizations require a module called [Folium](https://github.com/wrobstory/folium). It is not a part of the Anaconda distribution, so you need to install it before you continue. Open a terminal and run:

``
pip install folium
``

If the command completed successfully, you should be able to execute the first cell without errors.

In [ ]:
from IPython.display import HTML
import folium
import json
import networkx as nx
from folium import plugins

Below are visualization helpers needed to display the routes on a map.

In [ ]:
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
#     map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))



#Got from http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Polyline_text_path.ipynb


def add_lines(folium_map, list_of_lines, filename, line_color='blue', line_weight=5):
    """
    Draws a number of lines to the given map.
    
    An individual line consists of (lat, lon) coordinate pairs, which are connected by line segments.
    The argument `list_of_lines` should be thus be a list of lists of coordinate pairs. For instance,
    
    [[(55.65, 12.59), (55.79, 12.79)], [(55.20, 12.50), (55.22, 12.62)]]
    
    will draw a two line, each made up of a single line segment.
    """
    
    


    swap_multi_line_string = []
    for line in list_of_lines:
        swap_line = []
        for lat, lon in line:
            swap_line.append((lon, lat))
        swap_multi_line_string.append(swap_line)

    feature = {"type": "MultiLineString", "coordinates": swap_multi_line_string}
    with open(filename, 'w') as f:
        json.dump(feature, f)

        
    wind_line = folium.PolyLine(
    list_of_lines,
    weight=3,
    color=line_color
).add_to(folium_map)

## Basic visualization

More often than not, scientific programming involves juggling numbers and abstract concepts that are kind of hard to form a clear picture of in your head. Fortunately, though, maps are delightfully visual. Here we show you:

1. How you can display a map in IPython;
2. How to add markers, such as circles or tear drops, to the map; and
3. How to draw lines. 

At present the support for map drawing in IPython is a little more experimental than we would have liked. If you cannot get the examples below to work, please try it in another browser. We have tested the notebook in a recent versions of Chrome and Safari.

In [ ]:
# Creates a map centered at `location`, which is a coordinate pair in decimal degrees.
map_viz = folium.Map(location=[55.6632322, 12.5870087], zoom_start=15)
map_viz

In [ ]:
# Place a circle marker at DR byen
dr_byen_loc = (55.658105, 12.590940)

map_viz.add_child(folium.features.CircleMarker(dr_byen_loc, radius=5, color='red'))
map_viz

In [ ]:
# Draw a path between DR Byen, Islands Brygge Station, and Statens Seruminsitut
# Note calls to `add_lines` places files in your homework folder, which you must include the upload.
islands_brygge_loc = (55.663425, 12.585951)
statens_serum_loc = (55.666037, 12.590951)
add_lines(map_viz, [[dr_byen_loc, islands_brygge_loc, statens_serum_loc]], 'example_path.json')
map_viz

### Map data

You will be working with freely available map data from the collaborative [Open Street Map](http://www.openstreetmap.org/) service. We have exported a small part of the dataset covering the area around the southern campus of the University of Copenhagen. You will find this in the file `bryggen.osm`. 

Additionally, we provide a loader function `read_way_network` from the module `osm` which parses the Open Street Map data and returns a `networkx` graph. The nodes in the graph represent points on the map, and the edges are roads or, more generally, *ways* between these points.

In the cell below we read in the network and bind it to the variable `bryggen`. We also import the function `gc_dist`, which calculates the great-circle distance between two points on the map.

In [ ]:
from osm import read_way_network, gc_dist
bryggen = read_way_network("bryggen.osm")
type(bryggen)

### Exercise 1.1: Properties of nodes and edges (10 pts)

To get a feel for how the way network is represented in the graph, please retrieve **5** nodes from the `bryggen` graph at random (you can do it manually, setting five fixed numbers) and for each of them output the following: 

- Node id
- Data associated with the node
- IDs of outgoing edges
- Data associated with the outgoing edges

At this point we also advice you to go the provided `osm.py` file and familiarize yourself with the source code, especially how the class `Way` is defined.

In [ ]:
 #YOUR CODE HERE
print (nx.info(bryggen))
import numpy as np
poolnumber = bryggen.number_of_nodes()
#print (poolnumber)
print ("")
randomchoice = np.random.choice(poolnumber, size = 5)
for i in randomchoice:
    print ("Node ID: ", bryggen.nodes()[i]) #print node id
    print ("Location: ", bryggen.node[bryggen.nodes()[i]]) #print location
    #print (bryggen.node[bryggen.nodes()[i]].values())
    print ("Data: ", bryggen.out_edges(bryggen.nodes()[i],data = True))

### Exercise 1.2: Find nearest node (10 pts)

Pick a point as close to the center of Tietgen Kollegiet as possible and retrieve the coordinates. You may use Google Maps for this purpose, as explained [here](https://support.google.com/maps/answer/18539?hl=en).

Once you have the coordinates of the point, you should implement a function `find_nearest_node`, which takes the `networkx` graph as well as the latitude and longtitude of a point as arguments. The return value should be the the node id of the nearest node on the map, measured by great circle distance. In other words, the function should map from a set of coordinates to the nearest way intersection.

To verify that your function works as expected, place a circle marker in the middle of Tietgen Kollegiet and draw a line from that point to the way intersection. Display the map.

In [ ]:
# Your code here
#55.660800, 12.589695
tietgen_kollegiet = [55.660800, 12.589695]
map_nearest = folium.Map(location=tietgen_kollegiet, zoom_start=15)
map_nearest.add_child(folium.features.CircleMarker(tietgen_kollegiet, radius=5, color='blue'))
node_lat = []
node_lon = []
for i in randomchoice:
    node_lat.append(list(bryggen.node[bryggen.nodes()[i]].values())[0])
    node_lon.append(list(bryggen.node[bryggen.nodes()[i]].values())[1])
def find_nearest_node (graph, lat, lon):
    node_lat = []
    node_lon = []
    for i in range(bryggen.number_of_nodes()):
        node_lat.append(list(bryggen.node[bryggen.nodes()[i]].values())[0])
        node_lon.append(list(bryggen.node[bryggen.nodes()[i]].values())[1])
    
    dis = []
    for i in range(bryggen.number_of_nodes()):
        distance = gc_dist(lat, lon, node_lat[i], node_lon[i])
        dis.append((distance, i))
    #print (sorted(dis))
    #print (dis)
    nearest = (node_lat[sorted(dis)[0][1]], node_lon[sorted(dis)[0][1]])
    print (nearest)
    map_nearest.add_child(folium.features.CircleMarker(nearest, radius=5, color='red'))
    return nearest
    #return map_viz
find_nearest_node (bryggen, tietgen_kollegiet[0], tietgen_kollegiet[1])
map_nearest

### Exercise 1.3 (15 pts)

Find and draw the shortest path in number of nodes travelled from `start_loc` to `goal_loc`. Use the **depth-first** search algorithm programmed in class. You will have to modify it slightly.

In [ ]:
start_loc = (55.663811, 12.596087)
goal_loc = (55.665372, 12.578170)

creatdic = {}
for i in range(bryggen.number_of_nodes()):
    #print (i)
    creatdic[tuple(bryggen.node[bryggen.nodes()[i]].values())] = bryggen.nodes()[i]
#print (creatdic)

for key, value in creatdic.items():
    #print (key)
    if start_loc == key:
        print (value)
    if goal_loc == key:
        print (value)

#There are no nodes for the start and goal locations. Try to find the nearest nodes.
near_start = find_nearest_node (bryggen, start_loc[0], start_loc[1])
near_goal = find_nearest_node (bryggen, goal_loc[0], goal_loc[1])
for key, value in creatdic.items():
    #print (key)
    if near_start == key:
        print ("nearest start node:")
        print (value)
        nearest_start_node = value
    if near_goal == key:
        print ("nearest goal node:")
        print (value)
        nearest_goal_node = value

In [ ]:
def get_children(graph, node):
    children = []
    
    for tuples in bryggen.out_edges(node):
        children.append(tuples[1])
    return children

get_children(bryggen, nearest_start_node)
get_children(bryggen, nearest_goal_node)

In [ ]:
data = bryggen.out_edges(bryggen.nodes())
#print (data)
graph_nodes = {}
#key = np.unique([i[0] for i in data])
key = [i[0] for i in data]
#print (key)
len(key)
for each_node in key:
    values = []
    
    for pair in data:
        if pair[0] == each_node:
            values.append(pair[1])
    graph_nodes[each_node] = values
print (graph_nodes)

In [ ]:
#copied the code from the ipynb in the class
def dfs(graph, goal, start):
    to_visit = [start]    
    visited = set()
    while (len(to_visit) != 0):
        current_state = to_visit.pop(0)
        if current_state == goal:
            return current_state
        else:
            current_children = get_children(graph, current_state)
            visited.add (current_state)
            next_nodes = [node for node in current_children if node not in visited]
            to_visit = next_nodes + to_visit
    return None

print(dfs(graph_nodes, nearest_goal_node, nearest_start_node))

In [ ]:
#copied the code from the ipynb in the class
def dfs_path(graph, goal, start):
    current_path = [start]
    to_visit = [(start, current_path)]
    visited = set()
    while (len(to_visit) != 0):
        current_state, current_path = to_visit.pop(0)
        if current_state == goal:
            return current_path
        else:
            current_children = get_children(graph, current_state)
            visited.add (current_state)
            next_nodes = [(node, current_path+[node]) for node in current_children if node not in visited]
            to_visit = next_nodes + to_visit
    return None
re_dfs_path = dfs_path(graph_nodes, nearest_goal_node, nearest_start_node)
print(dfs_path(graph_nodes, nearest_goal_node, nearest_start_node))

In [ ]:
 #YOUR CODE HERE
#get location from nodes
path_loc = []
for node in re_dfs_path:
    path_loc.append(tuple(bryggen.node[node].values()))
#print (path_loc)
       
map_dfs = folium.Map(location=start_loc, zoom_start=15)
add_lines(map_dfs, [path_loc], 'example_path.json')
map_dfs.add_child(folium.features.CircleMarker(start_loc, radius=5, color='orange'))
map_dfs.add_child(folium.features.CircleMarker(goal_loc, radius=5, color='black'))
#nearest_start_loc = list(bryggen.node[nearest_start_node].values())

map_dfs

### Exercise 1.4 (5 pts)

Find and draw the shortest path in number of nodes travelled from `start_loc` to `goal_loc`. Use the **breadth-first** search algorithm programmed in class.

In [ ]:
 #YOUR CODE HERE
#copied the code from the ipynb in the class
def bfs_path(graph, goal, start):
    current_path = [start]
    to_visit = [(start, current_path)] 
    visited = set()
    while (len(to_visit) != 0):
        current_state, current_path = to_visit.pop(0)
        if current_state == goal:
            return current_path
        else:
            current_children = get_children(graph, current_state)
            visited.add (current_state)
            next_nodes = [(node, current_path+[node]) for node in current_children if node not in visited]
            to_visit = to_visit + next_nodes
    return None

re_bfs_path = bfs_path(graph_nodes, nearest_goal_node, nearest_start_node)
print(bfs_path(graph_nodes, nearest_goal_node, nearest_start_node))

bfs_path_loc = []
for node in re_bfs_path:
    bfs_path_loc.append(tuple(bryggen.node[node].values()))

map_bfs = folium.Map(location=start_loc, zoom_start=15)
add_lines(map_bfs, [bfs_path_loc], 'example_path.json')

map_bfs.add_child(folium.features.CircleMarker(start_loc, radius=5, color='orange'))
map_bfs.add_child(folium.features.CircleMarker(goal_loc, radius=5, color='black'))
#add the line between start and locations to the neareat nodes
nearest_start_loc = list(bryggen.node[nearest_start_node].values())
#add_lines(map_dfs, [start_loc, nearest_start_loc], 'example_path.json')
map_bfs

### Exercise 1.5 (10 pts)

Find and draw the shortest path in meters from `start_loc` to `goal_loc`. Use A\* search (including heuristic function).

You can use the A\* implementation available in `networkx`.

In [ ]:
#Here I applied several heuristic function to try to find the shortest path.
#Define the get_distance function to return the meters distance of path.
def get_distance(nodelist):
    dis =[]
    for i in range(len(nodelist)):
        if i + 1 < len(nodelist):
            dis.append(bryggen.edge[nodelist[i]][nodelist[i+1]]['dist'])

    return sum(dis)

#define get path function.
def get_path(nodelist):
    path = []
    for node in nodelist:
        path.append(tuple(bryggen.node[node].values()))
    return path

#Create the new graph with weighted edges
graph_weighted = []
for tuples in data:
    graph_weighted.append((tuples[0],tuples[1],{"weight": bryggen.edge[tuples[0]][tuples[1]]['dist']}))

G = nx.DiGraph(data=graph_weighted)
len(G.nodes())

#Create a map to show the different method.
test_map = folium.Map(location=start_loc, zoom_start=15)
test_map.add_child(folium.features.CircleMarker(start_loc, radius=5, color='orange'))
test_map.add_child(folium.features.CircleMarker(goal_loc, radius=5, color='black'))

In [ ]:
#define the heuristic function with different method.
#1. using bsf
def bsf_distance(neighbor, goal):
    path = bfs_path(graph = bryggen, goal = goal, start = neighbor)
    #for node in path:
    return get_distance(path)
        
bsf_method = nx.astar_path(G, nearest_start_node, nearest_goal_node, heuristic=bsf_distance, weight='weight')

#print (bsf_method)
add_lines(test_map, [get_path(bsf_method)], 'example_path.json')
get_distance(bsf_method)

In [ ]:
#2. using great circle distance
def gc_dist_heuristic(neighbor, goal):
    #dis = graph_nodes[start][goal]['dist']
    neighbor_lat = tuple(bryggen.node[neighbor].values())[0]
    neighbor_lon = tuple(bryggen.node[neighbor].values())[1]
    goal_lat = tuple(bryggen.node[goal].values())[0]
    goal_lon = tuple(bryggen.node[goal].values())[1]
    dis = gc_dist(neighbor_lat, neighbor_lon, goal_lat, goal_lon)
    return dis
gc_method = nx.astar_path(G, nearest_start_node, nearest_goal_node, heuristic=gc_dist_heuristic, weight='weight')
add_lines(test_map, [get_path(gc_method)], 'example_path.json')
get_distance(gc_method)

In [ ]:
#3. Using the shortest path.
def shortest_path(neighbor, goal):
    path = nx.shortest_path(G, neighbor, goal, weight='weight')
    return get_distance(path)

sp_method = nx.astar_path(G, nearest_start_node, nearest_goal_node, heuristic=shortest_path, weight='weight')
#sp_method = nx.astar_path(bryggen, nearest_start_node, nearest_goal_node, heuristic=shortest_path, weight='weight')
get_distance(sp_method)

In [ ]:
#use the shorest path function directly.
ss = nx.shortest_path(bryggen, nearest_start_node, nearest_goal_node, weight='dist')
print (ss)
get_distance(ss)

In [ ]:
test_map
#When I change the parameter of weight, the result will also change.
#However, the value of 1361.8861932124305 meters is the shortest path that I can find.


In [ ]:
# A* search
help(nx.astar_path)

# Your code here
